Chapter 10 : Performance Python


In [55]:
import random

In [56]:
def average_py(n):
    s=0
    for i in range(n):
        s += random.random()
    return s /n

In [57]:
n= 10000000

In [58]:
%time average_py(n)

CPU times: user 560 ms, sys: 989 μs, total: 561 ms
Wall time: 562 ms


0.4999944118679695

In [59]:
%time sum([random.random() for _ in range(n)])/n

CPU times: user 750 ms, sys: 162 ms, total: 912 ms
Wall time: 911 ms


0.5000397072440032

In [60]:
import numpy as np

In [61]:
def average_np(n):
    s = np.random.random(n) # draws the random numbers "all at once" (no python loop)
    return s.mean() # returns average value

In [62]:
%time average_np(n)

CPU times: user 84 ms, sys: 12 ms, total: 96 ms
Wall time: 94.7 ms


np.float64(0.500100215406323)

In [63]:
%timeit average_np(n)

78.8 ms ± 972 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [64]:
s= np.random.random(n)
s.nbytes # number of bytes used for the created ndarray object.

80000000

In [65]:
import numba

In [66]:
average_nb = numba.jit(average_py)

In [67]:
%time average_nb(n)
# The compiling happens during runtime, leading to some overhead.

CPU times: user 173 ms, sys: 3.02 ms, total: 176 ms
Wall time: 174 ms


0.4999723315948138

In [68]:
%time average_nb(n)
# From the second execution (with the same input data types), the execution is faster.

CPU times: user 134 ms, sys: 14 μs, total: 134 ms
Wall time: 133 ms


0.5000957025291074

In [69]:
%timeit average_nb(n)

109 ms ± 5.19 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [70]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [71]:
%%cython -a
import random
def average_cy1(int n):
    cdef int i
    cdef float s = 0
    for i in range(n):
        s+= random.random()
    return s /n

In [72]:
%time average_cy1(n)

CPU times: user 657 ms, sys: 1.14 ms, total: 659 ms
Wall time: 656 ms


0.5001589059829712

In [73]:
%timeit average_cy1(n)

671 ms ± 27.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [74]:
def is_prime(I):
    if I % 2 == 0: return False
    for i in range(3, int(I ** 0.5) + 1, 2):
        if I % i == 0: return False
    return True

In [75]:
n = int(1e8 + 3)
n

100000003

In [76]:
%time is_prime(n)

CPU times: user 32 μs, sys: 1 μs, total: 33 μs
Wall time: 36.7 μs


False

In [77]:
p1 = int(1e8 + 7)
p1

100000007

In [78]:
%time is_prime(p1)

CPU times: user 289 μs, sys: 0 ns, total: 289 μs
Wall time: 296 μs


True

In [79]:
p2 = 100109100129162907

In [80]:
p2.bit_length()

57

In [81]:
%time is_prime(p2)

CPU times: user 8.96 s, sys: 730 μs, total: 8.96 s
Wall time: 8.96 s


True

In [82]:
is_prime_nb = numba.jit(is_prime)

In [83]:
%time is_prime_nb(n)

CPU times: user 70.3 ms, sys: 1.01 ms, total: 71.3 ms
Wall time: 70.7 ms


False

In [84]:
%time is_prime_nb(n)

CPU times: user 11 μs, sys: 1 μs, total: 12 μs
Wall time: 14.3 μs


False

In [85]:
%time is_prime_nb(p2)

CPU times: user 489 ms, sys: 2 ms, total: 491 ms
Wall time: 488 ms


True

In [86]:
%%cython
def is_prime_cy1(I):
    if I % 2 == 0: return False
    for i in range(3, int(I ** 0.5) + 1, 2):
        if I % i == 0: return False
    return True

In [87]:
%timeit is_prime(p1)

259 μs ± 24.5 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [88]:
%timeit is_prime_cy1(p1)

158 μs ± 3.32 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [89]:
%%cython
def is_prime_cy2(long I): # Static declarations for the two variables I and i
    cdef long i
    if I % 2 == 0: return False
    for i in range(3, int(I ** 0.5) + 1, 2):
        if I % i == 0: return False
    return True

In [90]:
%timeit is_prime_cy2(p1)

16 μs ± 370 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [91]:
%time is_prime_nb(p2)

CPU times: user 588 ms, sys: 4.05 ms, total: 592 ms
Wall time: 591 ms


True

In [92]:
%time is_prime_cy2(p2)

CPU times: user 559 ms, sys: 48 μs, total: 559 ms
Wall time: 556 ms


True

Multiprocessing


In [93]:
import multiprocessing as mp

In [94]:
pool = mp.Pool(processes=4) # mp.Pool object is instantiated with multiple precesses.

In [95]:
%time pool.map(is_prime, 10*[p1]) # Then the respective function is mapped to a list object with prime numbers.

CPU times: user 0 ns, sys: 2.79 ms, total: 2.79 ms
Wall time: 2.5 ms


[True, True, True, True, True, True, True, True, True, True]

In [96]:
%time pool.map(is_prime_nb, 10*[p2])

CPU times: user 7.93 ms, sys: 5 ms, total: 12.9 ms
Wall time: 2.19 s


[True, True, True, True, True, True, True, True, True, True]

In [97]:
%time pool.map(is_prime_cy2, 10*[p2])

CPU times: user 14.1 ms, sys: 1.03 ms, total: 15.1 ms
Wall time: 2.36 s


[True, True, True, True, True, True, True, True, True, True]

Fibonacci Numbers

In [98]:
def fib_rec_py1(n):
    if n<2:
        return n
    else:
        return fib_rec_py1(n-1)+fib_rec_py1(n-2)

In [99]:
%time fib_rec_py1(35)

CPU times: user 1.79 s, sys: 941 μs, total: 1.79 s
Wall time: 1.79 s


9227465

In [100]:
fib_rec_nb = numba.jit(fib_rec_py1)

In [101]:
%time fib_rec_nb(35)

CPU times: user 13.6 ms, sys: 994 μs, total: 14.6 ms
Wall time: 14.3 ms


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Untyped global name 'fib_rec_py1': Cannot determine Numba type of <class 'function'>

File "../../../../../../tmp/ipykernel_3382/3187787909.py", line 5:
<source missing, REPL/exec in use?>

During: Pass nopython_type_inference

In [102]:
%%cython
def fib_rec_cy(int n):
    if n<2:
        return n
    else:
        return fib_rec_cy(n-1) + fib_rec_cy(n-2)

In [103]:
%time fib_rec_cy(35)

CPU times: user 679 ms, sys: 4.02 ms, total: 683 ms
Wall time: 678 ms


9227465

The major problem with the recursive algorithm is that intermediate results are not
cached but rather recalculated. To avoid this particular problem, a decorator can be
used that takes care of the caching of intermediate results. This speeds up the execu‐
tion by multiple orders of magnitude

In [104]:
from functools import lru_cache as cache


In [105]:
@cache(maxsize=None)
def fib_rec_py2(n):
    if n<2:
        return n
    else:
        return fib_rec_py2(n-1) + fib_rec_py2(n-2)

In [106]:
%time fib_rec_py2(35)

CPU times: user 24 μs, sys: 0 ns, total: 24 μs
Wall time: 25.3 μs


9227465

In [107]:
%time fib_rec_py2(80)

CPU times: user 27 μs, sys: 1 μs, total: 28 μs
Wall time: 30 μs


23416728348467685

In [112]:
%time fib_rec_py2(1000)

CPU times: user 8 μs, sys: 0 ns, total: 8 μs
Wall time: 11.2 μs


43466557686937456435688527675040625802564660517371780402481729089536555417949051890403879840079255169295922593080322634775209689623239873322471161642996440906533187938298969649928516003704476137795166849228875